In [1]:
import pandas as pd
import numpy as np

from scipy.stats.stats import pearsonr
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [143]:
# Load RNA Seq Data
rna_seq = pd.read_csv('/project/DSone/as3ek/data/csv_files/transcriptome_merged.csv')

/apps/software/standard/core/anaconda/5.2.0-py3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13468,13469,13470) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [144]:
rna_seq[rna_seq['Diagnosis'].isin(['CELIAC', 'CONTROL'])].reset_index(drop=True).head()

,Gene,PID,RNASes_Bx_loc,Diagnosis,A1BG,A1CF,A2M,A4GALT,A4GNT,AAAS,...,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,SEEM ID,Bx Date,Accession #,FileNames
0,2540002-9,C002,Doudenum,CELIAC,2.347019,25.421972,124.48038,1.980171,0.010000,18.622360,...,1.375860,5.907832,3.748820,32.835430,22.541649,11.531942,C-002,2017-04-13 00:00:00,GI-17-3505A,GI-17-3505A
1,2540004-9,C004,Doudenum,CELIAC,1.944927,27.633224,78.76862,1.473732,1.782160,13.545438,...,0.710877,3.942007,2.528200,25.566238,15.254350,6.072643,C-004,2017-05-01 00:00:00,GI-17-4060A,GI-17-4060A
2,2540031-9,C031,Doudenum,CELIAC,1.286928,23.610662,138.25372,2.306936,0.010000,13.497106,...,1.172150,4.238639,4.012072,25.441893,14.689348,9.913745,C-031,2018-08-02 00:00:00,GI-18-07450A,GI-18-07450A
3,2540032-9,C032,Doudenum,CELIAC,1.229353,22.671690,133.90640,1.742326,10.360900,12.895801,...,1.648440,5.001438,4.950277,21.775680,16.350965,13.422350,C-032,2018-08-07 00:00:00,GI-18-07655A,GI-18-07655A
4,2540033-9,C033,Doudenum,CELIAC,1.975784,22.597551,101.89975,1.012606,0.891341,10.065294,...,1.340800,3.710309,2.990033,21.703770,14.308658,9.205167,C-033,2018-08-10 00:00:00,GI-18-07807A,GI-18-07807A


In [3]:
# Load features from the CNN model
train_feat_max = pd.read_csv('/project/DSone/as3ek/data/csv_files/features_celiac__normal.csv')
valid_feat_max = pd.read_csv('/project/DSone/as3ek/data/csv_files/features_celiac__normal_valid.csv')

In [4]:
# Load cleaned EE RNA Seq Data
ee_rna_seq = pd.read_csv('/project/DSone/as3ek/data/csv_files/ee_trascript.csv')
ee_rna_seq = ee_rna_seq.drop(['Unnamed: 0'], axis=1)

In [8]:
# Load RNA Seq Data
rna_seq = pd.read_csv('/project/DSone/as3ek/data/csv_files/transcriptome_merged.csv')

# Drop unnecessary columns
rna_seq = rna_seq.drop(['Gene', 'PID', 'RNASes_Bx_loc', 'SEEM ID', 'Bx Date', 'Accession #'], axis=1)

# Filter all data to get only normal RNA Seq data
normal_rna_seq = rna_seq[rna_seq['Diagnosis'] == 'CONTROL'].reset_index(drop=True)
normal_rna_seq = normal_rna_seq.drop('Diagnosis', axis=1)

# Filter all data to get only normal RNA Seq data
celiac_rna_seq = rna_seq[rna_seq['Diagnosis'] == 'CELIAC'].reset_index(drop=True)
celiac_rna_seq = celiac_rna_seq.drop('Diagnosis', axis=1)

/apps/software/standard/core/anaconda/5.2.0-py3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13468,13469,13470) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
celiac_rna_seq.head()

,A1BG,A1CF,A2M,A4GALT,A4GNT,AAAS,AACS,AADAC,AAED1,AAGAB,...,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,FileNames
0,2.347019,25.421972,124.48038,1.980171,0.010000,18.622360,11.078631,124.08652,2.730297,17.697521,...,7.279240,8.349922,24.039724,1.375860,5.907832,3.748820,32.835430,22.541649,11.531942,GI-17-3505A
1,1.944927,27.633224,78.76862,1.473732,1.782160,13.545438,13.048814,257.55106,1.789949,27.142975,...,5.700451,6.408322,24.005035,0.710877,3.942007,2.528200,25.566238,15.254350,6.072643,GI-17-4060A
2,1.286928,23.610662,138.25372,2.306936,0.010000,13.497106,12.191676,101.12962,2.320212,24.512012,...,8.955390,10.433591,23.415869,1.172150,4.238639,4.012072,25.441893,14.689348,9.913745,GI-18-07450A
3,1.229353,22.671690,133.90640,1.742326,10.360900,12.895801,13.162550,276.89230,2.748407,20.668043,...,9.474550,7.819117,19.242214,1.648440,5.001438,4.950277,21.775680,16.350965,13.422350,GI-18-07655A
4,1.975784,22.597551,101.89975,1.012606,0.891341,10.065294,8.812082,169.54845,2.241079,16.227331,...,5.545079,5.369827,11.697079,1.340800,3.710309,2.990033,21.703770,14.308658,9.205167,GI-18-07807A


In [12]:
normal_rna_seq.head()

,A1BG,A1CF,A2M,A4GALT,A4GNT,AAAS,AACS,AADAC,AAED1,AAGAB,...,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,FileNames
0,1.527489,39.161606,141.393680,2.250741,3.340980,13.393648,11.521561,190.26294,3.802879,27.817423,...,7.966211,5.824094,15.380531,1.90331,6.084198,5.986877,19.513960,30.423355,13.065509,GI-17-2781A
1,1.836630,20.815561,70.397380,1.366245,8.056910,13.729294,11.792672,188.86170,2.597826,19.777740,...,6.067741,3.992341,12.340939,1.04346,4.722662,2.653590,24.902603,14.547653,10.382214,GI-17-5415A
2,2.070108,30.049267,78.129480,2.200778,8.668561,14.197790,13.160649,299.86722,2.576309,22.687897,...,6.487394,4.760917,12.637662,1.09973,5.131794,4.623030,23.566916,17.086746,8.982910,GI-17-5671A
3,2.323137,27.274525,93.843880,2.508640,1.156290,15.076123,11.050514,176.36838,1.392020,19.399889,...,4.988570,2.749776,13.213606,1.35546,5.087205,2.442820,26.665813,20.346804,7.571774,GI-17-6710A
4,1.550304,28.912386,78.544624,2.113191,0.260323,14.713420,10.754949,272.44060,2.326984,22.267065,...,7.437720,5.820460,18.061235,1.39190,5.067470,3.432585,24.128992,15.404562,10.151729,GI-17-6728A


In [14]:
name_map = pd.read_csv('/scratch/as3ek/histo_visual_recog/scripts/data/cchmc_mapping.csv')

In [30]:
feat_max = pd.concat([train_feat_max, valid_feat_max])

In [55]:
gi_feat_max = feat_max[feat_max['fname'].str.contains('GI')].reset_index(drop=True)
gi_feat_max['fname'] = gi_feat_max['fname'].str.split('__').str[0].str.split('_').str[0].str.replace('I', 'I-').str.replace(' ', '').str.replace('A1', 'A')

In [56]:
c_feat_max = feat_max[feat_max['fname'].str.contains('C')].reset_index(drop=True)
c_feat_max['fname'] = c_feat_max['fname'].str.split('__').str[0].str.split('_').str[0]

In [59]:
# Group features for all patches by biopsy name
gi_feat_max_grpd = gi_feat_max.groupby('fname', as_index=False).mean()
c_feat_max_grpd = c_feat_max.groupby('fname', as_index=False).mean()

In [62]:
c_feat_max_grpd.head()

,fname,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,C027,1.569540,1.800629,2.777871,1.160911,2.100492,1.973405,2.890815,3.700204,1.435636,...,2.146676,2.234218,1.676906,2.773962,1.852900,0.716745,1.483975,1.156939,1.616572,1.182015
1,C031,2.226476,2.614796,1.191848,2.356947,1.677903,1.381091,1.735029,2.460516,3.493864,...,2.044576,1.216872,1.455889,0.954203,3.254052,1.042613,2.279874,1.648835,0.992039,1.662386
2,C032,2.028006,2.937899,1.173732,2.243904,1.690223,1.474388,1.781492,2.154859,2.999383,...,2.040479,1.030623,1.442668,1.137877,3.027513,1.056706,2.569509,1.531834,0.929448,1.406503
3,C033,1.950046,3.059400,1.209982,2.162022,1.867939,2.222352,1.583083,1.621948,2.341786,...,1.905583,1.521226,1.300404,1.074216,2.650147,0.974185,2.618763,1.584212,1.017927,1.649350
4,C034,1.932541,3.410113,1.298151,2.051920,1.945213,1.997006,1.859203,2.207159,2.698378,...,2.181139,1.488641,1.476682,1.314644,2.687068,0.930547,2.500947,1.662968,1.039017,1.537389


In [63]:
gi_feat_max_grpd.head()

,fname,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,GI-17-10289A,1.473160,1.664941,2.161360,1.368585,1.817131,2.512023,1.800048,2.280767,1.295154,...,1.824156,3.185858,2.918280,1.852221,0.787371,0.480029,0.747601,1.087269,3.073834,0.817811
1,GI-17-10498A,1.917369,1.478766,1.801828,1.980441,1.442035,3.487439,1.396218,3.322703,1.369927,...,2.312621,3.988256,5.212091,1.797359,0.280776,0.429408,0.609097,1.044054,5.870791,0.747502
2,GI-17-10706A,1.687944,1.440563,2.734079,1.206448,1.704757,2.729145,1.734858,2.927566,1.081435,...,2.002050,3.381622,3.009862,1.818307,0.813056,0.562646,0.768400,1.185493,3.329921,0.846993
3,GI-17-2781A,1.495067,1.204786,2.941421,1.239465,1.391773,3.086098,1.855882,3.591309,1.278847,...,2.027716,3.655886,4.059050,2.225151,0.504351,0.408825,0.506742,0.872373,4.237109,1.067718
4,GI-17-3505A,1.744918,1.847216,1.341570,1.445397,2.693716,1.355879,1.372638,2.060323,1.648659,...,2.519777,2.549064,2.463550,0.919064,0.823797,0.913631,0.483297,1.921564,2.159932,1.290757


In [67]:
name_map = name_map[['PID', 'Accession #']]

In [75]:
# Merge seq with features
c_feat_max_grpd = pd.merge(c_feat_max_grpd, name_map, left_on='fname', right_on='PID')

In [79]:
c_feat_max_grpd = c_feat_max_grpd.drop(['fname', 'PID'], axis=1)

In [81]:
c_feat_max_grpd = c_feat_max_grpd.rename(columns={'Accession #':'fname'})

In [90]:
feat_max_grpd = pd.concat([gi_feat_max_grpd, c_feat_max_grpd], ignore_index=True, sort=False)
feat_max_grpd.to_csv('/scratch/as3ek/histo_visual_recog/results/image_features_celiac_normal_clean.csv', index=False)

In [91]:
rna_seq = pd.concat([normal_rna_seq, celiac_rna_seq], ignore_index=True, sort=False)
rna_seq.to_csv('/scratch/as3ek/histo_visual_recog/results/rna_seq_celiac_normal_clean.csv', index=False)

In [95]:
# Merge seq with features
seq_feat = pd.merge(rna_seq, feat_max_grpd, left_on='FileNames', right_on='fname')
seq_feat.to_csv('/scratch/as3ek/histo_visual_recog/results/rnaseq_feat_celiac_normal.csv', index=False)

In [98]:
# Getting list of all columns for correlation b/w
x = list(rna_seq)
x.remove('FileNames')

y = list(feat_max_grpd)
y.remove('fname')

In [99]:
# Calculate pairwise correlation and filter it for reqd columns and round the values
corr = seq_feat.corr()
corr_fltrd = corr[x][corr.index.isin(y)]
corr_fltrd = corr_fltrd.round(4)

In [100]:
corr_fltrd.to_csv('/scratch/as3ek/histo_visual_recog/results/gene_feat_corr_celiac_normal.csv', index=False)

In [105]:
gene_map = pd.read_csv('/scratch/as3ek/histo_visual_recog/scripts/data/custom.txt', sep='\t')

In [106]:
def corr_cutoff(corr_fltrd, cutoff):
    # Make copy
    df = corr_fltrd.copy().T
    
    # Replace all values below cutoff with 0
    for col in y:
        df.loc[np.abs(df[col]) < cutoff, col] = 0
    
    # Remove all columns with all zeros (Remove unnecessary features)
    df = df.loc[:, (df != 0).any(axis=0)]
    
    # Transpose and remove all columns with all zeros (Remove unnecessary genes)
    df = df.T
    df = df.loc[:, (df != 0).any(axis=0)]
    
    return df

In [107]:
def compute_corr_metrics(corr_fltrd_cutoff, seq_feat, gene_map):
    corr_metrics = pd.DataFrame(columns=['Gene Symbol', 'Gene Name', 'Feature', 'PearsonR', 'p-value'])
    c = corr_fltrd_cutoff.copy()
    for gene in c.columns:
        df = pd.DataFrame(c[gene]).T
        df = df.loc[:, (df != 0).any(axis=0)]
        
        names = list(gene_map[gene_map['Approved symbol'] == gene]['Approved name'])
        if len(names) > 0:
            name = names[0]
        else:
            name = gene
        
        for feat in df.columns:
            r, p = pearsonr(seq_feat[feat], seq_feat[gene])
            tmp = pd.DataFrame([[gene, name, feat, r, p]], columns=['Gene Symbol', 'Gene Name', 'Feature', 'PearsonR', 'p-value'])
            corr_metrics = pd.concat([corr_metrics, tmp], ignore_index=True)
    return corr_metrics

In [108]:
def compute_regulation(normal_rna_seq, ee_rna_seq, genes, cutoff_fold, gene_map):
    regulation_data = pd.DataFrame(columns=['Regulation', 'Gene Symbol', 'Gene Name', 'fold'])
    for col in genes:
        a = np.mean(ee_rna_seq[col]) / np.mean(normal_rna_seq[col]) 
        b = 1 / a
        names = list(gene_map[gene_map['Approved symbol'] == col]['Approved name'])
        if len(names) > 0:
            name = names[0]
        else:
            name = col
        if np.abs(a) > cutoff_fold:
            tmp = pd.DataFrame([['up', col, name, a]], columns=['Regulation', 'Gene Symbol', 'Gene Name', 'fold'])
            regulation_data = pd.concat([regulation_data, tmp], ignore_index=True)
        if np.abs(b) > cutoff_fold:
            tmp = pd.DataFrame([['down', col, name, b]], columns=['Regulation', 'Gene Symbol', 'Gene Name', 'fold'])
            regulation_data = pd.concat([regulation_data, tmp], ignore_index=True)
    
    return regulation_data

In [109]:
regulation_data = compute_regulation(normal_rna_seq, ee_rna_seq, x, 1, gene_map)

In [116]:
regulation_data.to_csv('/scratch/as3ek/histo_visual_recog/scripts/data/gene_regulation_data_celiac_normal.csv', index=False)

In [130]:
corr_fltrd_cutoff = corr_cutoff(corr_fltrd, 0.7)

In [ ]:
plt.figure(figsize=(30, 200))
sns.heatmap(corr_fltrd_cutoff)

In [203]:
corr_fltrd_cutoff.head()

,AAMDC,ABCA5,ABCG1,ACBD4,ACOX2,ACSF2,ADAMTS13,ADAMTSL5,ADCK5,AKR7A3,...,UBAC2,UPB1,VAMP2,VIPR1,WDTC1,YPEL3,YPEL5,ZFAND2B,ZFYVE19,ZFYVE28
5,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0
8,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0
17,0.0,0.0,0.0,-0.8252,0.0,0.0,0.0,0.0,-0.8021,0.0,...,0.0,0.0,0.0,-0.8338,0.0,-0.8149,0.0,-0.8425,-0.8179,0.0
19,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0
36,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0


In [132]:
corr_metrics = compute_corr_metrics(corr_fltrd_cutoff, seq_feat, gene_map)
corr_metrics.to_csv('/scratch/as3ek/histo_visual_recog/scripts/data/corr_metric_celiac_normal_cutoff_8.csv', index=False)

In [210]:
corr_metrics = pd.read_csv('/scratch/as3ek/histo_visual_recog/scripts/data/corr_metric_celiac_normal_cutoff_7.csv')

In [211]:
corr_metrics.head()

,Gene Symbol,Gene Name,Feature,PearsonR,p-value
0,AAMDC,adipogenesis associated Mth938 domain containing,8,-0.737954,3.683384e-08
1,AAMDC,adipogenesis associated Mth938 domain containing,15,-0.719808,1.133104e-07
2,AAMDC,adipogenesis associated Mth938 domain containing,16,0.717032,1.335456e-07
3,AAMDC,adipogenesis associated Mth938 domain containing,17,-0.734327,4.644255e-08
4,AAMDC,adipogenesis associated Mth938 domain containing,21,-0.733887,4.775661e-08


In [212]:
corr_metrics_grpd = corr_metrics[['Gene Symbol', 'Gene Name', 'Feature']].groupby(by='Gene Symbol', as_index=False).agg({'Gene Name': 'first',
                                                                                                     'Feature': 'count'})

corr_metrics_grpd.columns = ['Gene Symbol', 'Gene Name', 'Number of Features']

In [213]:
corr_metrics_grpd.to_csv('/scratch/as3ek/histo_visual_recog/scripts/data/corr_metric_grouped_celiac_normal_cutoff_7.csv', index=False)

In [214]:
corr_metrics_grpd

,Gene Symbol,Gene Name,Number of Features
0,AAMDC,adipogenesis associated Mth938 domain containing,209
1,AATK,apoptosis associated tyrosine kinase,134
2,ABAT,4-aminobutyrate aminotransferase,6
3,ABCA10,ATP binding cassette subfamily A member 10,120
4,ABCA5,ATP binding cassette subfamily A member 5,142
...,...,...,...
1710,ZSCAN16,zinc finger and SCAN domain containing 16,15
1711,ZSCAN26,zinc finger and SCAN domain containing 26,87
1712,ZSCAN30,zinc finger and SCAN domain containing 30,169
1713,ZSWIM5,zinc finger SWIM-type containing 5,1


In [152]:
from sklearn.cluster import KMeans

In [205]:
kmeans_all_data = KMeans(n_clusters=5, random_state=0).fit(corr_fltrd)
kmeans_cutoff_data = KMeans(n_clusters=5, random_state=0).fit(df_cl)

In [206]:
corr_fltrd['cluster'] = kmeans_all_data.labels_
df_cl['cluster'] = kmeans_cutoff_data.labels_

In [219]:
feat_cluster = pd.DataFrame()

for cluster in np.unique(df_cl['cluster']):
    cl = df_cl[df_cl['cluster'] == cluster].drop('cluster', axis=1).copy().T
    
    # Remove all 0 values from columns
    cl = cl.loc[:, (cl != 0).any(axis=0)]

    # Remove all 0 values from rows
    cl = cl[(cl.T != 0).any()]
    
    tmp = pd.DataFrame([cluster, list(cl.index), len(cl.index), list(cl.columns), len(cl.columns)]).T
    feat_cluster = pd.concat([feat_cluster, tmp])

In [221]:
feat_cluster.columns = ['cluster', 'genes', 'num_genes', 'features', 'num_features']

In [222]:
feat_cluster.to_csv('/scratch/as3ek/histo_visual_recog/results/feature_clusters_cutoff_8.csv', index=False)

In [226]:
np.min(corr_fltrd['DUOXA2'])

-0.44

In [235]:
np.mean(normal_rna_seq[np.unique(sgnfct_seq_feat_corr['gene'])[0]])

4035.613696

In [234]:
np.mean(ee_rna_seq[np.unique(sgnfct_seq_feat_corr['gene'])[0]])

2174.0717784313724

In [233]:
np.unique(sgnfct_seq_feat_corr['gene'])[0]

array(['ALDOB', 'AMACR', 'APOH', 'CA2', 'CAPN13', 'CDK20', 'CHAD',
       'CRIP3', 'CYP4F3', 'EPHX1', 'HGD', 'IGSF9', 'ISX', 'IYD', 'KCNK10',
       'KIF12', 'LTK', 'MAPKAPK3', 'ME1', 'MMP24', 'MMP28', 'MOCS1',
       'NELL2', 'OAT', 'PPP1R36', 'RAPGEFL1', 'RGS11', 'SEMA6C', 'SHMT1',
       'SLC5A4', 'TMEM229A', 'TMEM25', 'TMEM52', 'UNC5D', 'UNC93A',
       'UPB1', 'USP2'], dtype=object)

In [236]:
normal_rna_seq[np.unique(sgnfct_seq_feat_corr['gene'])[0]]

0     3672.1885
1     4264.6830
2     3090.2253
3     4540.7686
4     3200.6025
5     3714.6306
6     5261.3057
7     4697.8080
8     6668.3076
9     2550.5593
10    4364.0550
11    4898.0825
12    6355.1100
13    5055.7250
14    3362.0535
15    2191.2117
16    2555.9316
17    4323.9400
18    4143.9870
19    5283.7720
20    3237.6394
21    3434.0540
22    3716.8413
23    3360.2893
24    2946.5710
Name: ALDOB, dtype: float64

In [125]:
corr_fltrd

,A1BG,A1CF,A2M,A4GALT,A4GNT,AAAS,AACS,AADAC,AAED1,AAGAB,...,ZUFSP,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.3119,-0.3472,0.1445,-0.0967,0.3854,-0.1643,0.1294,-0.4289,-0.0932,0.1329,...,-0.3946,0.1735,0.4780,0.5078,-0.2422,-0.4870,-0.2796,-0.1401,-0.5111,-0.3466
1,0.1284,-0.2042,0.1863,-0.3056,0.1060,-0.0096,0.2321,-0.2797,-0.2290,0.0615,...,-0.3042,0.1258,0.2579,0.3757,-0.2778,-0.3434,-0.1957,0.0560,-0.3595,-0.2769
2,-0.3313,0.3809,-0.1416,0.0465,-0.1911,-0.1501,-0.2423,0.4914,0.2693,-0.0293,...,0.4153,-0.0648,-0.3472,-0.4914,0.3739,0.4142,0.3454,-0.2233,0.5010,0.3856
3,0.3825,-0.3396,0.1219,0.0234,0.2292,0.0570,0.2309,-0.4308,-0.2162,0.0134,...,-0.3335,0.1083,0.3519,0.3605,-0.1833,-0.2411,-0.2442,0.0422,-0.4327,-0.2243
4,-0.1607,0.0719,0.1798,-0.2205,-0.0421,-0.0450,0.0937,0.0601,-0.0293,-0.0024,...,-0.0585,0.1563,0.1940,0.1574,0.1914,0.0768,0.1835,-0.0511,0.1600,0.1300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,0.2917,-0.4330,0.3311,-0.1984,0.3086,-0.0923,0.2482,-0.4771,-0.1667,0.0517,...,-0.4201,0.2521,0.4859,0.5534,-0.1840,-0.4522,-0.1614,-0.0778,-0.4755,-0.2524
2044,0.0656,-0.2723,0.3559,-0.2524,0.0765,-0.0987,0.1507,-0.3535,-0.2127,-0.0640,...,-0.3267,0.2501,0.3119,0.3026,-0.1424,-0.3544,-0.1184,-0.0337,-0.4055,-0.1784
2045,0.1669,-0.3481,0.3248,-0.2280,0.1997,-0.0506,0.1406,-0.4249,-0.2032,0.0453,...,-0.3808,0.2210,0.4234,0.4981,-0.2388,-0.4771,-0.1805,-0.0162,-0.4239,-0.2731
2046,-0.0881,0.2946,-0.3856,0.3001,-0.1771,0.1268,-0.1719,0.3417,0.1009,0.0299,...,0.3229,-0.2371,-0.3443,-0.3795,0.1240,0.3586,0.0411,0.0833,0.3025,0.1483
